In [4]:
import os
import numpy as np
import mne

%matplotlib widget

In [5]:
sample_data_folder = mne.datasets.sample.data_path()
sample_data_raw_file = os.path.join(sample_data_folder, 'MEG', 'sample', 'sample_audvis_raw.fif')

In [6]:
raw = mne.io.read_raw_fif(sample_data_raw_file)
raw.crop(tmax=60).load_data()

Opening raw data file /Users/markyousef/mne_data/MNE-sample-data/MEG/sample/sample_audvis_raw.fif...
    Read a total of 3 projection items:
        PCA-v1 (1 x 102)  idle
        PCA-v2 (1 x 102)  idle
        PCA-v3 (1 x 102)  idle
    Range : 25800 ... 192599 =     42.956 ...   320.670 secs
Ready.
Current compensation grade : 0
Reading 0 ... 36037  =      0.000 ...    60.000 secs...


<Raw  |  sample_audvis_raw.fif, n_channels x n_times : 376 x 36038 (60.0 sec), ~107.0 MB, data loaded>

In [7]:
raw.copy().pick_types(meg=False, stim=True).plot(start=3, duration=6);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Converting a STIM channel signal to an Events array

In [8]:
events = mne.find_events(raw, stim_channel='STI 014')

86 events found
Event IDs: [ 1  2  3  4  5 32]


## Reading embedded events as Annotations

In [10]:
testing_data_folder = mne.datasets.testing.data_path()
eeglab_raw_file = os.path.join(testing_data_folder, 'EEGLAB', 'test_raw.set')

Using default location ~/mne_data for testing...
[............................................................] 100.00% (964.1 MB,   4.4 MB/s) \   
Verifying hash 347e5b73864f3fda7324b0e2fc8747d3.
Decompressing the archive: /Users/markyousef/mne_data/mne-testing-data-0.65.tar.gz
(please be patient, this can take some time)
Successfully extracted to: ['/Users/markyousef/mne_data/MNE-testing-data']


In [11]:
eeglab_raw = mne.io.read_raw_eeglab(eeglab_raw_file)

Reading /Users/markyousef/mne_data/MNE-testing-data/EEGLAB/test_raw.fdt


In [12]:
eeglab_raw.annotations

<Annotations  |  154 segments : square (80), rt (74), orig_time : None>

In [13]:
len(eeglab_raw.annotations)

154

In [14]:
set(eeglab_raw.annotations.duration)

{0.0}

In [15]:
set(eeglab_raw.annotations.description)

{'rt', 'square'}

In [16]:
eeglab_raw.annotations.onset[0]

1.000068359375

## Converting between Events arrays and Annotations objects

From Annotations Object to Events Array

In [17]:
events_from_annot, event_dict = mne.events_from_annotations(eeglab_raw)

Used Annotations descriptions: ['rt', 'square']


In [18]:
event_dict

{'rt': 1, 'square': 2}

In [19]:
events_from_annot[:10]

array([[ 128,    0,    2],
       [ 217,    0,    2],
       [ 267,    0,    1],
       [ 602,    0,    2],
       [ 659,    0,    1],
       [ 987,    0,    2],
       [1372,    0,    2],
       [1447,    0,    1],
       [1757,    0,    2],
       [1807,    0,    1]])

In [20]:
custom_mapping = {'rt': 77, 'square': 42}
(events_from_annot, event_dict) = mne.events_from_annotations(eeglab_raw, event_id=custom_mapping)

Used Annotations descriptions: ['rt', 'square']


In [21]:
event_dict

{'rt': 77, 'square': 42}

In [22]:
events_from_annot[:10]

array([[ 128,    0,   42],
       [ 217,    0,   42],
       [ 267,    0,   77],
       [ 602,    0,   42],
       [ 659,    0,   77],
       [ 987,    0,   42],
       [1372,    0,   42],
       [1447,    0,   77],
       [1757,    0,   42],
       [1807,    0,   77]])

From Events Array to Annotations Object

In [24]:
mapping = {1: 'auditory/left', 2: 'auditory/right', 3: 'visual/left', 4: 'visual/right', 5: 'smiley', 32: 'buttonpress' }
onsets = events[:, 0] / raw.info['sfreq']
durations = np.zeros_like(onsets) # assumes instantaneous events
descriptions = [mapping[event_id] for event_id in events[:, 2]]

In [26]:
annot_from_events = mne.Annotations(onset=onsets, duration=durations, description=descriptions, orig_time=raw.info['meas_date'])

In [27]:
raw.set_annotations(annot_from_events)

<Raw  |  sample_audvis_raw.fif, n_channels x n_times : 376 x 36038 (60.0 sec), ~107.1 MB, data loaded>

In [28]:
raw.plot(start=5, duration=5);

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## Making multiple events per annotation

Create REM annotations

In [29]:
rem_annot = mne.Annotations(onset=[5, 41], duration=[16, 11], description=['REM'] * 2)

In [30]:
raw.set_annotations(rem_annot)

<Raw  |  sample_audvis_raw.fif, n_channels x n_times : 376 x 36038 (60.0 sec), ~107.1 MB, data loaded>

In [31]:
(rem_events, rem_event_dict) = mne.events_from_annotations(raw, chunk_duration=1.5)

Used Annotations descriptions: ['REM']


In [32]:
np.round((rem_events[:, 0] - raw.first_samp) / raw.info['sfreq'], 3)

array([ 5.   ,  6.5  ,  8.   ,  9.5  , 11.   , 12.501, 14.001, 15.501,
       16.999, 18.499, 41.   , 42.5  , 44.   , 45.5  , 47.   , 48.5  ,
       50.   ])